### Ai Earth Engine 《官方文档》 - 应用案例
- 典型植被指数计算及区域统计
> 对检索的影像（以 Landsat-8 为例），通过波段运算计算常见的指数。并以归一化植被指数（ NDVI ）为例，进行区域均值统计以及时序折线图制作。

In [1]:
import aie
# 使用token进行鉴权
aie.Authenticate(token='a1d1b9fec8d44f88343352e117bbd983')
# 使用 aliyun access_key_id进行鉴权
# aie.Authenticate(access_key_id="*", access_key_secret="*")
# 使用 aliyun RAM STS 进行鉴权(需要 aiearth-core>=1.0.3)
# aie.Authenticate(access_key_id='*', access_key_secret='*', access_key_security_token='*')
aie.Initialize()

计算资源初始化中，请等待...
计算资源初始化完成.


#### 典型光谱指数算法
> 定义典型指数计算方法。使用 aie.Image.add 、 aie.Image.subtract 、 aie.Image.multiply 和 aie.Image.divide 实现影像波段运算。另外可使用 aie.Image.normalizedDifference 实现两个波段的归一化差值运算 (Band1-Band2)/(Band1+Band2) ，使用 aie.Image.expression 可实现构建表达式对影像进行波段运算。
- 如切换卫星数据源，需要调整对应的波段名称

In [2]:
# 比值植被指数 (RVI, Ratio Vegetation Index）
# RVI常用于监测植被覆盖、评估生长状态或估算生物量，但因其对土壤背景敏感，可能受非植被区域干扰。
def getRVI(image):
    nir = image.select(['SR_B5'])   # 提取近红外波段（SR_B5）
    red = image.select(['SR_B4'])   # 提取红波段（SR_B4）
    rvi = nir.divide(red)      # 计算 RVI = 近红外 / 红波段
    return rvi.rename(['RVI'])  #重命名影像波段名称

# 增强型植被指数 （Enhanced Vegetation Index，EVI）
# 克服了传统植被指数（如 NDVI）在高植被覆盖度区域饱和的问题，能够更敏感地反映植被的生长状态。
def getEVI(image):
    evi = image.expression(
        '(2.5 * (nir - red)) /(nir + 6 * red - 7.5 * blue + 1)',      #通过构建表达式对影像进行波段运算
        {
            'nir': image.select(['SR_B5']),
            'red': image.select(['SR_B4']),
            'blue': image.select(['SR_B2'])
    }).rename(['EVI'])
    return evi

# 归一化植被指数 （Normalized Difference Vegetation Index，NDVI）
# NDVI = (NIR - Red) / (NIR + Red)， 用于评估植被的光合作用和植被覆盖度
def getNDVI(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename(['NDVI'])  #计算两个波段的归一化差值
    return ndvi

# 近红外植被指数    (Near-Infrared Vegetation Index，NIRv）
# NIRv=近红外波段（nir）* NDVI
def getNIRv(image):
    nir = image.select(['SR_B5'])
    nirv = nir.multiply(image.normalizedDifference(['SR_B5', 'SR_B4'])).rename(['NIRv'])
    return nirv

# 土壤调整植被指数   (Soil-Adjusted Vegetation Index，SAVI）
#  SAVI = (NIR - Red) * L / (NIR + Red + C)     植被指数，减少了土壤背景对植被指数的影响
def getSAVI(image):
    nir = image.select(['SR_B5'])
    red = image.select(['SR_B4'])
    savi = ((nir.subtract(red)).multiply(aie.Image.constant(1.5))).divide((nir.add(red)).add(aie.Image.constant(0.5))).rename(['SAVI'])
    return savi

# 归一化水体指数  （Normalized Difference Water Index，NDWI）
#NDWI = (Green - NIR) / (Green + NIR)  用于监测水体的指数，范围通常在 -1 到 1 之间，正值表示可能存在水体，负值表示可能是植被或土壤
def getNDWI(image):
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename(['NDWI'])
    return ndwi

#### Landsat-8 数据检索
- 指定区域、时间、云量检索 Landsat-8 ，并对数据进行去云处理。
- 矢量区域裁选遥感图像区域
- 定义去云和云阴影算法 

In [3]:
region = aie.FeatureCollection('China_Province') \
            .filter(aie.Filter.eq('province', '浙江省'))

In [4]:
def l8Collection(startdate, enddate):
    images = aie.ImageCollection('LANDSAT_LC08_C02_T1_L2') \
            .filterBounds(region) \
            .filterDate(startdate, enddate)
    return images
def removeLandsatCloud(image):
    cloudShadowBitMask = (1 << 4)
    cloudsBitMask = (1 << 3)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(aie.Image(cloudShadowBitMask)).eq(aie.Image(0)).And(qa.bitwiseAnd(aie.Image(cloudsBitMask)).eq(aie.Image(0)))
    return image.updateMask(mask)

In [5]:
lc8_collection = l8Collection('2021-08-01', '2021-08-31')
lc8_collection.map(removeLandsatCloud)
print(lc8_collection.size().getInfo())

19


- 在整个影像集合上计算最大值合成影像，这通常用来获得无云或少云的最佳视图
> .max()是应用于影像集合（Image Collection）的一个方法，用于从集合中的所有影像中计算一个合成影像。具体来说，它会在每个像素位置（即每一个地理坐标）上选择具有最大值的像素来生成一张新的影像

In [6]:
lc8_img = lc8_collection.max()

#### 指数图层的可视化
- 以 NDVI 计算为例输出指数计算成果，并地图可视化展示

In [9]:
ndvi = getNDVI(lc8_img)
ndvi_vis = {
    'min': -0.2,
    'max': 0.6,
    'palette': [
        '#2B83BA', '#ABDDA4', '#FFFFBF', '#FDAE61', '#D7191C'
    ]
}

map = aie.Map(
    center=ndvi.getCenter(),
    height=800,
    zoom=5
)

map.addLayer(
    ndvi,
    ndvi_vis,
    'NDVI',
    bounds=ndvi.getBounds()
)
map

Map(center=[29.58457794645941, 120.62290153913469], controls=(ZoomControl(options=['position', 'zoom_in_text',…

#### NDVI 区域统计

> 使用中国市级行政区划数据，按照市域范围对 NDVI 进行均值统计。使用 aie.Image.reduceRegions 和 aie.Reducer.mean 实现对影像进行指定区域范围均值统计。 当在较大范围内执行 ReduceRegion 或者 ReduceRegions 函数时，可能存在较为耗时的情况。开发者根据实际需求调整 scale（单位：米），scale 越大，耗时越少。

In [10]:
ndvi=getNDVI(lc8_img)
zone = aie.FeatureCollection('China_City') \
          .filter(aie.Filter.eq('province', '浙江省'))
# 计算浙江省内每个子区域的 NDVI 平均值。
# reduceRegions 方法用于在一个或多个区域上对影像进行聚合操作.
region_mean = ndvi.reduceRegions(zone, aie.Reducer.mean(), 1000)
region_info = region_mean.getInfo()

x_list = []
y_list = []
for feature in region_info['features']:
    x_list.append(feature['properties']['city'])
    y_list.append(feature['properties']['NDVI_mean'])
# print(x_list)
# print(y_list)

from bqplot import pyplot as plt
plt.figure(1, title='2021年浙江省各市NDVI均值统计')
plt.bar(x_list, y_list)   #colors=['MediumSeaGreen']
plt.show()

#### NDVI时间序列分析
- 在指定空间范围内实现时间序列统计分析，并绘制折线图。

In [14]:
# 构建时间序列函数
def doSeries(start_time, end_time, zone):
    lc8_col = l8Collection(start_time, end_time)
    lc8_col.map(removeLandsatCloud)
    lc8_img = lc8_col.mosaic()
    ndvi = getNDVI(lc8_img)
    return ndvi.reduceRegion(aie.Reducer.mean(), zone, 1000)

In [15]:
# 融合矢量集合中所有矢量对象的几何对象，以Geometry数据结构返回融合后的几何对象
zone = aie.FeatureCollection('China_Province') \
          .filter(aie.Filter.eq('province', '浙江省')).geometry()

x_ndvi_series = []
y_ndvi_series = []

year = '2021'
mon = ['01','02','03','04','05','06','07','08','09','10','11','12']
lday = ['31','28','31','30','31','30','31','31','30','31','30','31']

for i in range(0,12):
    startdate = year + '-' + mon[i] + '-01'
    enddate = year + '-' + mon[i] + '-' + lday[i]
    lc8_ndvi_mon = doSeries(startdate, enddate , zone)
    x_ndvi_series.append(mon[i] + '月')
    y_ndvi_series.append(lc8_ndvi_mon.getInfo()['NDVI_mean'])

# print(x_ndvi_series)
# print(y_ndvi_series)
from bqplot import pyplot as plt
plt.figure(2, title='2021年浙江省逐月NDVI均值统计')
plt.plot(x_ndvi_series, y_ndvi_series)
plt.show()

#### 导出图像（离线计算）

In [ ]:
task = aie.Export.image.toAsset(ndvi, 'NDVI', 30)
task.start()